In [ ]:
!pip install -q -U kaggle_environments

In [68]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Стратегия 1: Всегда выбирает "камень" (значение 0) на каждом ходу.

Описание: Очень простая и предсказуемая стратегия. Агент не реагирует на действия противника и не использует никакой информации о предыдущих ходах.

In [69]:
%%writefile rock_agent.py

def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Стратегия 2: Всегда выбирает "бумагу" (значение 1) на каждом ходу.

Описание: Очень простая и предсказуемая стратегия. Агент не реагирует на действия противника и не использует никакой информации о предыдущих ходах.

In [70]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1  # 1 соответствует "бумага"

Overwriting paper_agent.py


Стратегия 3: Всегда выбирает "ножницы" (значение 2) на каждом ходу.

Описание: Очень простая и предсказуемая стратегия. Агент не реагирует на действия противника и не использует никакой информации о предыдущих ходах.

In [71]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2  # 2 соответствует "ножницы"

Overwriting scissors_agent.py


Стратегия 4: На каждом ходу случайно выбирает одно из возможных действий: "камень", "бумагу" или "ножницы".

Описание: Использует генератор случайных чисел для выбора действия. Поведение агента непредсказуемо для противника.

In [72]:
%%writefile random_agent.py

import random

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


Стратегия 5: Последовательно перебирает действия в порядке: "камень" $\rightarrow$ "бумага" $\rightarrow$ "ножницы" $\rightarrow$ повтор.

Описание: Использует текущий номер хода (observation.step) для определения следующего действия по модулю количества знаков.

In [73]:
%%writefile cycle_agent.py

def cycle_agent(observation, configuration):
    return observation.step % configuration.signs

Overwriting cycle_agent.py


Стратегия 6: Выбирает действие, которое побеждает последнее действие противника.

Описание: Если противник на прошлом ходу выбрал "камень", агент выберет "бумагу", чтобы победить.

In [74]:
%%writefile beat_last_move_agent.py

def beat_last_move_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return 0  # Начинает с "камня"

Overwriting beat_last_move_agent.py


Стратегия 7: Выбирает действие, которое проигрывает последнему действию противника.

Описание: Если противник выбрал "бумагу", агент выберет "камень", который проиграет "бумаге".

In [75]:
%%writefile lose_to_last_move_agent.py

def lose_to_last_move_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return 0

Overwriting lose_to_last_move_agent.py


Стратегия 8: Отслеживает, какое действие противник использует чаще всего, и выбирает действие, которое побеждает его.

Описание: Считает количество каждого действия противника и выбирает контрдействие против самого частого.

In [76]:
%%writefile frequency_agent.py

def frequency_agent(observation, configuration):
    import numpy as np
    if observation.step == 0:
        frequency_agent.counts = [0] * configuration.signs
    else:
        frequency_agent.counts[observation.lastOpponentAction] += 1
    most_common = frequency_agent.counts.index(max(frequency_agent.counts))
    return (most_common + 1) % configuration.signs

Overwriting frequency_agent.py


Стратегия 9: Повторяет последнее действие противника.

Описание: Если противник на прошлом ходу выбрал "ножницы", агент тоже выберет "ножницы".

In [77]:
%%writefile copy_opponent.py

import random

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Стратегия 10: Отслеживает, какое действие противник использует реже всего, и выбирает действие, которое побеждает его.

Описание: Нацелен на использование слабостей в игре противника, предполагая, что редко используемые действия могут быть неожиданными.

In [78]:
%%writefile anti_frequency_agent.py

def anti_frequency_agent(observation, configuration):
    import numpy as np
    if observation.step == 0:
        anti_frequency_agent.counts = [0] * configuration.signs
    else:
        anti_frequency_agent.counts[observation.lastOpponentAction] += 1
    least_common = anti_frequency_agent.counts.index(min(anti_frequency_agent.counts))
    return (least_common + 1) % configuration.signs

Overwriting anti_frequency_agent.py


Стратегия 11: Выбирает действие, которое побеждает его собственное предыдущее действие.

Описание: Если агент на прошлом ходу выбрал "камень", то на следующем выберет "бумагу", чтобы победить "камень".

In [ ]:
%%writefile mirror_agent.py

import random

def mirror_agent(observation, configuration):
    global my_last_action
    if observation.step == 0:
        # На первом шаге выбираем случайное действие и сохраняем его
        action = random.randrange(0, configuration.signs)
        my_last_action = action
    else:
        # Выбираем действие, которое побеждает наше предыдущее действие
        action = (my_last_action + 1) % configuration.signs
        # Обновляем наше последнее действие для следующего хода
        my_last_action = action
    return action

Overwriting mirror_agent.py


Стратегия 12: Начинает с "камня", затем всегда повторяет предыдущее действие противника.

Описание: Стратегия основана на взаимности и копировании поведения противника после первого хода.

In [80]:
%%writefile tit_for_tat_agent.py

def tit_for_tat_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return 0  # Начинает с "камня"


Overwriting tit_for_tat_agent.py


Стратегия 13: Чаще выбирает "камень" по сравнению с другими действиями.

Описание: Вероятность выбора действий распределена неравномерно: "камень" — 50%, "бумага" — 25%, "ножницы" — 25%.

In [81]:
%%writefile biased_random_agent.py

import random

def biased_random_agent(observation, configuration):
    choices = [0] * 50 + [1] * 25 + [2] * 25  # 50% камень, 25% бумага, 25% ножницы
    return random.choice(choices)

Overwriting biased_random_agent.py


Стратегия 14: Следует заранее определённой последовательности действий, например: ["камень", "бумага", "ножницы", "камень", "ножницы", "бумага"].

Описание: Повторяет эту последовательность циклически, независимо от действий противника.

In [82]:
%%writefile fixed_sequence_agent.py

def fixed_sequence_agent(observation, configuration):
    sequence = [0, 1, 2, 0, 2, 1]  # Предопределенная последовательность
    return sequence[observation.step % len(sequence)]

Overwriting fixed_sequence_agent.py


Стратегия 15: Предсказывает следующее действие противника на основе его предыдущих действий с использованием матрицы переходов (Марковской цепи).

Описание: Строит статистическую модель поведения противника и выбирает действие, которое побеждает ожидаемый следующий ход противника.

In [83]:
%%writefile markov_agent.py

import random

def markov_agent(observation, configuration):
    if observation.step == 0:
        markov_agent.prev_action = None
        markov_agent.transition_matrix = [[1/3]*3 for _ in range(3)]
        return random.randrange(0, configuration.signs)
    else:
        if markov_agent.prev_action is not None:
            last = markov_agent.prev_action
            curr = observation.lastOpponentAction
            markov_agent.transition_matrix[last][curr] += 1
            total = sum(markov_agent.transition_matrix[last])
            probs = [count / total for count in markov_agent.transition_matrix[last]]
            predicted = probs.index(max(probs))
            action = (predicted + 1) % configuration.signs
        else:
            action = random.randrange(0, configuration.signs)
        markov_agent.prev_action = observation.lastOpponentAction
        return action

Overwriting markov_agent.py


После того, как все агенты созданы, можно организовать турнир между ними. Для этого используется функция evaluate.

In [94]:
# Список имен агентов и соответствующих им файлов
agent_files = [
    ("Rock Agent", "rock_agent.py"),
    ("Paper Agent", "paper_agent.py"),
    ("Scissors Agent", "scissors_agent.py"),
    ("Random Agent", "random_agent.py"),
    ("Cycle Agent", "cycle_agent.py"),
    ("Beat Last Move Agent", "beat_last_move_agent.py"),
    ("Copy Opponent Agent", "copy_opponent.py"),
    ("Lose to Last Move Agent", "lose_to_last_move_agent.py"),
    ("Frequency Agent", "frequency_agent.py"),
    ("Anti-Frequency Agent", "anti_frequency_agent.py"),
    ("Mirror Agent", "mirror_agent.py"),
    ("Tit-for-Tat Agent", "tit_for_tat_agent.py"),
    ("Biased Random Agent", "biased_random_agent.py"),
    ("Fixed Sequence Agent", "fixed_sequence_agent.py"),
    ("Markov Agent", "markov_agent.py"),
]

# Инициализируем таблицу результатов
agent_names = [name for name, _ in agent_files]
results = pd.DataFrame(0, index=agent_names, columns=agent_names)

# Запускаем турниры между всеми парами агентов
for i, (name1, file1) in enumerate(agent_files):
    for j, (name2, file2) in enumerate(agent_files):
        if i != j:
            outcomes = evaluate(
                "rps",
                [file1, file2],
                configuration={"episodeSteps": 100, "tieRewardThreshold": 1},
                num_episodes=10
            )
            # Извлекаем средний результат для первого агента
            score = outcomes[0][0]
            results.loc[name1, name2] = score

# Выводим таблицу результатов
display(results)

,Rock Agent,Paper Agent,Scissors Agent,Random Agent,Cycle Agent,Beat Last Move Agent,Copy Opponent Agent,Lose to Last Move Agent,Frequency Agent,Anti-Frequency Agent,Mirror Agent,Tit-for-Tat Agent,Biased Random Agent,Fixed Sequence Agent,Markov Agent
Rock Agent,0,-99,99,2,0,-98,1,98,-99,97,0,0,1,0,-96
Paper Agent,99,0,-99,-2,0,-97,-1,99,-98,0,0,1,10,0,-96
Scissors Agent,-99,99,0,2,0,-99,0,97,-97,99,0,-1,-27,0,-98
Random Agent,16,-4,-18,0,2,-12,-11,0,0,0,-1,2,-4,-5,-8
Cycle Agent,0,0,0,-2,0,0,99,-98,0,-99,0,98,-10,0,-1
Beat Last Move Agent,98,97,99,-6,0,0,50,-49,49,96,-1,49,14,-48,32
Copy Opponent Agent,0,1,1,-16,-99,99,0,50,99,0,-99,1,-1,-2,-34
Lose to Last Move Agent,-98,-99,-97,-12,98,49,-49,0,-99,-19,98,-49,-11,50,-96
Frequency Agent,99,98,97,-3,0,-49,21,99,0,98,0,21,28,0,-40
Anti-Frequency Agent,-97,0,-99,-3,99,-96,-1,19,-98,0,99,99,-6,83,-97


Анализ полученных результатов:

In [96]:
# Подсчитываем общее количество очков для каждого агента
total_scores = results.sum(axis=1)
# Сортируем по убыванию
total_scores = total_scores.sort_values(ascending=False)

print("Итоговые результаты турнира:")
print(total_scores)

Итоговые результаты турнира:
Markov Agent               597
Beat Last Move Agent       480
Frequency Agent            469
Mirror Agent               229
Copy Opponent Agent          0
Cycle Agent                -13
Random Agent               -43
Biased Random Agent        -54
Fixed Sequence Agent       -55
Rock Agent                 -94
Anti-Frequency Agent       -98
Scissors Agent            -124
Paper Agent               -184
Lose to Last Move Agent   -334
Tit-for-Tat Agent         -355
dtype: int64


Лидеры турнира:

    Markov Agent - Предиктивные стратегии, основанные на статистическом анализе поведения противника, оказываются наиболее эффективными.

    Beat Last Move Agent - Простые адаптивные стратегии, которые непосредственно контрят предыдущие действия противника, также эффективны.

    Frequency Agent - Стратегии, использующие частотный анализ, эффективны против агентов с предсказуемым или несбалансированным поведением.